### Libraries

In [6]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os                              
import pandas as pd                     
import sys                              
sys.path.append(os.path.abspath('..'))
from src import utils
import numpy as np
import datetime
import re

'machine learning library imports'
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Load data

In [7]:
data_dir = os.path.abspath('../data')

# Set the desired option ('None', 'SMOTE', 'SMOTE-NC', 'RealTabFormer', 'GReaT')
AUGMENTATION = 'RealTabFormer'
# Conditional structure to load the appropriate dataset based on the AUGMENTATION option
if AUGMENTATION == 'None':
    df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k.csv'), low_memory=False)
elif AUGMENTATION == 'SMOTE':
    df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE.csv'), low_memory=False)
elif AUGMENTATION == 'SMOTE-NC':
    df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_SMOTE_NC.csv'), low_memory=False)
elif AUGMENTATION == 'RealTabFormer':
    df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_RealTabFormer.csv'), low_memory=False)
elif AUGMENTATION == 'GReaT':
    df_train = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_train_100k_GReaT.csv'), low_memory=False)
else:
    raise ValueError("AUGMENTATION option not recognized. Please choose between 'None', 'SMOTE', 'SMOTE-NC', 'RealTabFormer', or 'GReaT'.")

'Test data for all datasets'
df_test = pd.read_csv(os.path.join(data_dir, 'EdgeIIot_test.csv'), low_memory=False)

MemoryError: Unable to allocate 11.4 MiB for an array with shape (1500000,) and data type int64

### Data preparation

In [ ]:
# Drop columns mbtcp.unit_id and mbtcp.trans_id from train and test data    
df_train = df_train.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)
df_test = df_test.drop(['mbtcp.unit_id', 'mbtcp.trans_id'], axis=1)

# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [13]:
if AUGMENTATION == 'None':
    # Concatenate X_train and X_test to ensure consistent encoding for both
    concatenated_data = pd.concat([X_train, X_test], axis=0)

    # Extract categorical features
    categorical_features = concatenated_data.select_dtypes(include="object").columns

    # Get the unique values of all categorical columns
    for col in concatenated_data[categorical_features].columns:
        unique_values = concatenated_data[col].unique()
        print(f'{col}: \n{unique_values}\n')

    # One-hot encode categorical features for both X_train and X_test
    concatenated_data_encoded = pd.get_dummies(concatenated_data, 
                                               columns=categorical_features, 
                                               drop_first=True, 
                                               dtype='int8')

    # Split the data back into X_train and X_test
    X_train = concatenated_data_encoded.iloc[:len(X_train)]
    X_test = concatenated_data_encoded.iloc[len(X_train):]
    
    print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')
    
    def get_unique_values(df, columns):
        unique_values = df[columns].apply(lambda x: ''.join(x.astype(str)), axis=1).unique()
        unique_values.sort()
        return unique_values

    http_referer_columns = [col for col in X_train.columns if 'http.referer' in col]
    http_request_version_columns = [col for col in X_train.columns if 'http.request.version' in col]
    dns_qry_name_len_columns = [col for col in X_train.columns if 'dns.qry.name.len' in col]
    mqtt_conack_flags_columns = [col for col in X_train.columns if 'mqtt.conack.flags' in col]
    mqtt_protoname_columns = [col for col in X_train.columns if 'mqtt.protoname' in col]
    http_request_method_columns = [col for col in X_train.columns if 'http.request.method' in col]

    unique_values_http_referer = get_unique_values(X_train, http_referer_columns)
    unique_values_http_request_version = get_unique_values(X_train, http_request_version_columns)
    unique_values_dns_qry_name_len = get_unique_values(X_train, dns_qry_name_len_columns)
    unique_values_mqtt_conack_flags = get_unique_values(X_train, mqtt_conack_flags_columns)
    unique_values_mqtt_protoname = get_unique_values(X_train, mqtt_protoname_columns)

    # Apply the same function to X_test
    unique_values_http_referer_test = get_unique_values(X_test, http_referer_columns)
    unique_values_http_request_version_test = get_unique_values(X_test, http_request_version_columns)
    unique_values_dns_qry_name_len_test = get_unique_values(X_test, dns_qry_name_len_columns)
    unique_values_mqtt_conack_flags_test = get_unique_values(X_test, mqtt_conack_flags_columns)
    unique_values_mqtt_protoname_test = get_unique_values(X_test, mqtt_protoname_columns)
    
    # Join the unique values from both train and test data and print them
    unique_values_http_referer = np.unique(np.concatenate((unique_values_http_referer, unique_values_http_referer_test)))
    unique_values_http_request_version = np.unique(np.concatenate((unique_values_http_request_version, unique_values_http_request_version_test)))
    unique_values_dns_qry_name_len = np.unique(np.concatenate((unique_values_dns_qry_name_len, unique_values_dns_qry_name_len_test)))
    unique_values_mqtt_conack_flags = np.unique(np.concatenate((unique_values_mqtt_conack_flags, unique_values_mqtt_conack_flags_test)))
    unique_values_mqtt_protoname = np.unique(np.concatenate((unique_values_mqtt_protoname, unique_values_mqtt_protoname_test)))

    print(f'Unique values for http.referer: \n{unique_values_http_referer}\n')
    print(f'Unique values for http.request.version: \n{unique_values_http_request_version}\n')
    print(f'Unique values for dns.qry.name.len: \n{unique_values_dns_qry_name_len}\n')
    print(f'Unique values for mqtt.conack.flags: \n{unique_values_mqtt_conack_flags}\n')
    print(f'Unique values for mqtt.protoname: \n{unique_values_mqtt_protoname}\n')
    

    # check if X_train and X_test have categorical features
    print(f'X_train categorical features: {X_train.select_dtypes(include="object").columns}')
    print(f'X_test categorical features: {X_test.select_dtypes(include="object").columns}')

http.request.method: 
['0.0' '0' 'GET' 'POST' 'TRACE' 'OPTIONS' 'SEARCH' 'PROPFIND' 'PUT']

http.referer: 
['0.0' '0' '127.0.0.1'
 '() { _; } >_[$($())] { echo 93e4r0-CVE-2014-6278: true; echo;echo; }'
 'TESTING_PURPOSES_ONLY']

http.request.version: 
['0.0' '0' 'HTTP/1.1' 'HTTP/1.0' 'script>alert(1)/script><\\" HTTP/1.1'
 '/etc/passwd|?data=Download HTTP/1.1'
 '-al&ABSOLUTE_PATH_STUDIP=http://cirt.net/rfiinc.txt?? HTTP/1.1'
 '-al&_PHPLIB[libdir]=http://cirt.net/rfiinc.txt?? HTTP/1.1' '-a HTTP/1.1'
 'Src=javascript:alert(\'Vulnerable\')><Img Src=\\" HTTP/1.1'
 "name=a><input name=i value=XSS>&lt;script>alert('Vulnerable')</script> HTTP/1.1"
 'By Dr HTTP/1.1' '> HTTP/1.1']

dns.qry.name.len: 
['0.0' '1.0' '0' '2.debian.pool.ntp.org' '1.debian.pool.ntp.org'
 '3.debian.pool.ntp.org' '0.debian.pool.ntp.org' 'raspberrypi.local'
 'null-null.local' '_googlecast._tcp.local']

mqtt.conack.flags: 
['0.0' '0' '0x00000000' '1574358' '1461589' '1461383' '1574359' '1461074'
 '1461384' '1461591']

mq

#### Columns name validation for XGBoost

In [14]:
# define a regular expression pattern to match (),:; {}><[]$- 
pattern = r'[(),:;{}><\[\]\$\-]|_{2,}|\s+'

# clean X_train  column names using the regular expression pattern
cleaned_columns = [re.sub(pattern, '', col) for col in X_train.columns]

# check for duplicates and if they exist, add suffix to make them unique
unique_names = []
for name in cleaned_columns:
    if name in unique_names:
        # add suffix to make the name unique
        suffix = 1
        while f"{name}_{suffix}" in unique_names:
            suffix += 1
        unique_names.append(f"{name}_{suffix}")
    else:
        unique_names.append(name)

# Replace the column names with the cleaned and unique names
X_train.columns, X_test.columns = unique_names, unique_names

# print the unique feature names
print(unique_names)

# check if any of X_train column names have [, ] or <, > characters
for name in X_train.columns:
    if '[' in name or ']' in name or '<' in name or '>' in name:
        print(name)

['arp.opcode', 'arp.hw.size', 'icmp.checksum', 'icmp.seq_le', 'icmp.unused', 'http.content_length', 'http.response', 'http.tls_port', 'tcp.ack', 'tcp.ack_raw', 'tcp.checksum', 'tcp.connection.fin', 'tcp.connection.rst', 'tcp.connection.syn', 'tcp.connection.synack', 'tcp.flags', 'tcp.flags.ack', 'tcp.len', 'tcp.seq', 'udp.stream', 'udp.time_delta', 'dns.qry.name', 'dns.qry.qu', 'dns.qry.type', 'dns.retransmission', 'dns.retransmit_request', 'dns.retransmit_request_in', 'mqtt.conflag.cleansess', 'mqtt.conflags', 'mqtt.hdrflags', 'mqtt.len', 'mqtt.msg_decoded_as', 'mqtt.msgtype', 'mqtt.proto_len', 'mqtt.topic_len', 'mqtt.ver', 'mbtcp.len', 'http.request.method_0.0', 'http.request.method_GET', 'http.request.method_OPTIONS', 'http.request.method_POST', 'http.request.method_PROPFIND', 'http.request.method_PUT', 'http.request.method_SEARCH', 'http.request.method_TRACE', 'http.referer_0', 'http.referer_0.0', 'http.referer_127.0.0.1', 'http.referer_TESTING_PURPOSES_ONLY', 'http.request.version

In [81]:
# # Extract categorical features
# categorical_features = X_train.select_dtypes(include="object").columns

# # Concatenate X_train and X_test
# X_comb = pd.concat([X_train[categorical_features], X_test[categorical_features]], axis=0)

# # Apply one-hot encoding (get_dummies)
# X_comb_enc = pd.get_dummies(X_comb, dtype='int8')

# # Split back into X_train and X_test
# X_train_enc, X_test_enc = train_test_split(
#     X_comb_enc, test_size=len(X_test), random_state=42)

# # Print the shape of X_train_enc and X_test_enc
# print(f'X_train_enc shape: {X_train_enc.shape}, X_test_enc shape: {X_test_enc.shape}')

X_train_enc shape: (536515, 53), X_test_enc shape: (381934, 53)


In [82]:
# # Drop columns categorical_features from X_train and X_test 
# X_train = X_train.drop(categorical_features, axis=1)
# X_test = X_test.drop(categorical_features, axis=1)

# # Concatenate X_train and X_train_enc
# X_train = pd.concat([X_train.reset_index(drop=True), X_train_enc.reset_index(drop=True)], axis=1)
# X_test = pd.concat([X_test.reset_index(drop=True), X_test_enc.reset_index(drop=True)], axis=1)

# # Print the shape of X_train and X_test
# print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}')

X_train shape: (536515, 90), X_test shape: (381934, 90)


#### Label encoding

In [15]:
# instantiate the label encoder
le = LabelEncoder()

# fit and encode the training labels
y_train = le.fit_transform(y_train)

# encode the test labels
y_test = le.transform(y_test)

print('Attack_type and encoded labels:\n')
for i, label in enumerate(le.classes_):
    print(f'{label:23s} {i:d}')

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


### Model training

In [16]:
# Shuffle training data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Instantiate model with 10 decision trees
xgb = XGBClassifier(max_depth=8,
                    learning_rate=0.1,
                    n_estimators=10,
                    verbosity=1,
                    silent=None,
                    #objective="multi:softmax",
                    objective="multi:softprob",
                    booster='gbtree',
                    n_jobs=1,
                    nthread=None,
                    gamma=0,
                    min_child_weight=1,
                    max_delta_step=0,
                    subsample=0.7,
                    colsample_bytree=1,
                    colsample_bylevel=1,
                    colsample_bynode=1,
                    reg_alpha=0,
                    reg_lambda=1,
                    base_score=0.5,
                    random_state=42,
                    seed=42,
                    num_class= (le.classes_).size)

# Train the model
xgb_clf =  xgb.fit(X_train, y_train)

### Model Evaluation

In [17]:
predictions = xgb_clf.predict(X_test)

In [18]:
accuracy = metrics.accuracy_score(y_test, predictions)
precision_w = metrics.precision_score(y_test, predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(y_test, predictions, average='weighted')
f1_score_w = metrics.f1_score(y_test, predictions, average='weighted')
precision_m = metrics.precision_score(y_test, predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(y_test, predictions, average='macro')
f1_score_m = metrics.f1_score(y_test, predictions, average='macro')

print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision (macro): {:.2f}".format(precision_m))
print("Recall (macro): {:.2f}".format(recall_m))
print("F1 (macro): {:.2f}".format(f1_score_m))
print("Precision (weighted): {:.2f}".format(precision_w))
print("Recall (weighted): {:.2f}".format(recall_w))
print("F1 (weighted): {:.2f}".format(f1_score_w))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")

Model Evaluation Metrics
~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: 0.98
Precision (macro): 0.93
Recall (macro): 0.91
F1 (macro): 0.92
Precision (weighted): 0.99
Recall (weighted): 0.98
F1 (weighted): 0.98
~~~~~~~~~~~~~~~~~~~~~~~~~


#### Save Metrics Results 

In [19]:
# create dictionary for results
results = {
    "model": "XGBoost",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/xgboost.csv')

#### Confusion Matrix

In [20]:
conf_mat = metrics.confusion_matrix(y_test, predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'

# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4682,0,0,0,1,0,0,0,0,81,14,0,0,4,0
DDoS_HTTP,0,9337,0,0,0,0,0,0,0,0,0,0,0,34,257
DDoS_ICMP,0,0,13451,0,4,46,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,24,0,17,0,1,81,0,0,0,23,0,0,0,0,0
MITM,0,0,0,0,0,0,71,0,0,0,0,0,5,0,0
Normal,0,0,0,0,0,0,0,272776,0,0,0,0,0,0,0
Password,0,0,0,0,0,0,0,0,9208,0,0,7,893,0,0
